# Do Vectordb

In [ ]:
# !pip install llama-index
# !pip install chromadb
# !pip install --upgrade llama-index
# !pip install --upgrade llama-index llama-index-vector-stores-chroma

# from llama_index.vector_stores.chroma import ChromaVectorStore



In [79]:
# Importing necessary libraries for loading datasets, data manipulation, document processing, vector storage, and embeddings.
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
from llama_index.core import Document, StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceWindowNodeParser
import chromadb
from llama_index.core.node_parser import TokenTextSplitter
from utils import chunked_iterable, load_config
from llama_index.vector_stores.chroma import ChromaVectorStore
import openai
import os
from utils import *

In [49]:
# Hardcoded values for easy adjustment
CHUNK_SIZE = 1000 #only for db upload
TOKEN_CHUNK_SIZE = 512
CHUNK_OVERLAP = 50

# Load the config file
load_config()
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key
hf_read_token = os.getenv("HF_READ_TOKEN")
from huggingface_hub import login
login(hf_read_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Hassan\.cache\huggingface\token
Login successful


In [ ]:
# # get hugging face token and log in.
# # load_dotenv()
# load_config()
# hf_read_token = os.getenv("HF_READ_TOKEN")
# # print(f"Token is: {hf_read_token}")

In [ ]:
# # Hardcoded values for easy adjustment
CHUNK_SIZE = 1000 #only for db upload
TOKEN_CHUNK_SIZE = 512
CHUNK_OVERLAP = 50

# Load the config file
load_config()
openai.api_key = os.getenv("OPENAI_API_KEY")
# !huggingface-cli login --add-to-git-credential
# # Load dataset and convert to DataFrame for easier manipulation
# dataset = load_dataset("jamescalam/ai-arxiv")
# # dataset = Dataset.load_from_disk("dataset1/aragog_data/ai_aragog1")
# df = pd.DataFrame(dataset['train'])


In [66]:
# from datasets import get_dataset_config_names

# # This will show the cache location
# from datasets.utils.logging import get_verbosity
# print(f"Cache directory: {get_verbosity()}")
# from datasets import config

# # Display the cache directory
# print(f"Cache directory: {config.HF_DATASETS_CACHE}")
# dataset.column_names
# dataset.save_to_disk('dataset1')
# df_dataset = pd.DataFrame(dataset)
# df_dataset.__len__
# !huggingface-cli login --add-to-git-credential
# dataset2 = load_dataset("jamescalam/ai-arxiv")

^C


ValueError: Invalid pattern: '**' can only be an entire path component

In [61]:
# do not use
# from datasets import load_from_disk
# dataset1 = load_from_disk('dataset1')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Hassan/AppData/Local/Temp/tmpfzi9l8d8/dataset1/state.json'

In [43]:
# read when getting dataset as pandas and storing locally
# instead of reading dataset, just read the pandas and save it because that is what is used
# import pandas as pd

# df_archiv = pd.read_json("hf://datasets/jamescalam/ai-arxiv/train.jsonl", lines=True)
# data_path = './dataset1/archiv_data.csv'
# df_archiv.to_csv(data_path, index=False)


In [72]:
# for debugging
# data_path = './dataset1/archiv_data.csv'
# df_archiv_local = pd.read_csv(data_path)
# df_archiv_local.columns

Index(['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment',
       'journal_ref', 'primary_category', 'published', 'updated', 'content',
       'references'],
      dtype='object')

In [73]:
# Prepare document objects from the dataset for indexing
# documents = [Document(text=content) for content in df['content']]
documents = [Document(text=content) for content in df_archiv_local['content']]

In [74]:
# Setup the embedding model
embed_model = OpenAIEmbedding(model="text-embedding-3-large")

chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [76]:
# Classic vector DB
# Initialize a text splitter with hardcoded values for chunking documents
parser = TokenTextSplitter(chunk_size=TOKEN_CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
nodes = parser.get_nodes_from_documents(documents)


# # chroma_collection = chroma_client.create_collection("ai_arxiv_full")
# chroma_collection = chroma_client.get_collection("ai_arxiv_full")

# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex(
#     nodes, storage_context=storage_context,
#     embed_model=embed_model,
#     use_async=True
# )


In [77]:
import nest_asyncio
nest_asyncio.apply()

# Continue with your existing code
chroma_collection = chroma_client.get_collection("ai_arxiv_full")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    use_async=True
)


In [81]:
# Sentence window
node_parser_sentence_window = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
nodes_sentence_window = node_parser_sentence_window.get_nodes_from_documents(documents)
nodes_sentence_window_filtered = filter_large_nodes(nodes_sentence_window)

# chroma_collection_sentence_window = chroma_client.create_collection("ai_arxiv_sentence_window")

chroma_collection_sentence_window = chroma_client.create_collection(
    "ai_arxiv_sentence_window", get_or_create=True
)

vector_store_sentence_window = ChromaVectorStore(chroma_collection=chroma_collection_sentence_window)

storage_context_sentence_window = StorageContext.from_defaults(vector_store=vector_store_sentence_window)


In [82]:
# Removed nodes longer than 8K, they were causing issues and these are only tables and references which are not relevant,
# in total 17 chunks or something
index = VectorStoreIndex(
    nodes_sentence_window_filtered,
    storage_context=storage_context_sentence_window,
    embed_model=embed_model,
    use_async=True,
    show_progress=True
)

Generating embeddings: 100%|██████████| 20/20 [00:03<00:00,  5.13it/s]


In [88]:
# Document summary index
# LLM (gpt-3.5-turbo)
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import DocumentSummaryIndex, get_response_synthesizer

parser = TokenTextSplitter(chunk_size=3072, chunk_overlap=100)
splits_doc_summary = parser.get_nodes_from_documents(documents)

docs_for_summary = [Document(text=node.text, metadata=node.metadata) for node in splits_doc_summary]

# from llama_index.llms.mistralai import MistralAI

In [ ]:
# # !pip show llama-index
# # !pip install llama-index-llms-mistral
# # !pip install llama-index-llm-mistral
# !pip install llama-index mistralai


In [89]:
from llama_index.llms import MistralAI

ImportError: cannot import name 'MistralAI' from 'llama_index.llms' (unknown location)